# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment, Dataset, Model
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import json
import requests

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'exp_iris_automl'

experiment=Experiment(ws, experiment_name)

In [ ]:
iris = load_iris()

# Combine features and target into a single DataFrame
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

# Register the dataset
datastore = ws.get_default_datastore()
iris_dataset = Dataset.Tabular.register_pandas_dataframe(
    name='iris_dataset',
    dataframe=iris_df,
    target=(datastore, 'iris_dataset')
)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "iteration_timeout_minutes": 5,
    "experiment_timeout_hours": 0.25,
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
    "enable_onnx_compatible_models": True
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='classification',
    training_data=iris_dataset,
    label_column_name='target',
    compute_target='notebook260045',
    path='.',
    **automl_settings
)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = remote_run.get_output()
print("Best Model: ", best_run)
print("Best Model Details: ", fitted_model)

In [ ]:
#TODO: Save the best model
model = best_run.register_model(model_name='automl_best_model', model_path='outputs/model.pkl')

In [ ]:
env = best_run.get_environment()

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Retrieve the environment used in the AutoML run
env = best_run.get_environment()

# Inference configuration
inference_config = InferenceConfig(entry_script="score_automl.py", environment=env)

# Deploy the model
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service_name = "service-automl"

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)
print(service.state)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
test_sample = json.dumps({'data': [iris.data[0].tolist()]})
headers = {'Content-Type': 'application/json'}

response = requests.post(service.scoring_uri, data=test_sample, headers=headers)
print(response.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
